In [ ]:
import torch
D_in, H, D_out= 784, 100, 10 #percenton multicapa con 783 inputs 100 capas y 10 outpus


model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),
)



In [ ]:
'sacando outputs'

outputs = model(torch.rand(64,784))   #064 imagenes con sus inputs
outputs.shape

torch.Size([64, 10])

In [ ]:
torch.cuda.is_available()

True

In [ ]:
#enviando a la GPU

model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
import sklearn
from sklearn.datasets import fetch_openml

#descarga de datos

mnist= fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

((70000, 784), (70000,))

In [ ]:
import numpy as np

#normalizacion y spliteo

X_train, X_test, Y_train, Y_test=  X[:60000]/255., X[60000:]/255.,Y[:60000].astype(np.int), Y[60000:].astype(np.int)

In [ ]:
#funcion de perdida y derivada
def softmax(x):
  return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

criterion= torch.nn.CrossEntropyLoss()

In [ ]:
#convirtiendo los datos y pasandolos a tensores
X_t= torch.from_numpy(X_train).float().cuda()
Y_t= torch.from_numpy(Y_train).long().cuda()


#bucle de fit
epochs = 100
lr= 0.8
log_each=10
l=[]


for e in range(1, epochs+1):

  #foward
  y_pred= model(X_t)

  #loss
  loss = cross_entropy(y_pred, Y_t)
  l.append(loss.item())

  #set zero grad
  model.zero_grad()

  #calculando todo los gradiendes automaticamente
  loss.backward()

  # update of weigths
  optimizer= torch.optim.SGD(model.parameters(), lr=0.8)

  if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 0.17652
Epoch 20/100 Loss 0.17652
Epoch 30/100 Loss 0.17652
Epoch 40/100 Loss 0.17652
Epoch 50/100 Loss 0.17652
Epoch 60/100 Loss 0.17652
Epoch 70/100 Loss 0.17652
Epoch 80/100 Loss 0.17652
Epoch 90/100 Loss 0.17652
Epoch 100/100 Loss 0.17652


In [ ]:
#evaluar el modelo recordando que l nn aveces se comporta raro para prede y para entrenar 
from sklearn.metrics import accuracy_score

def evaluate(x):
  model.eval()
  y_pred=model(x)
  y_probas = softmax(y_pred)
  return torch.argmax(y_probas, axis=1)


y_pred= evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(Y_test,y_pred.cpu().numpy())


0.9509